In [ ]:
#@title Скачиваем тексты песен исполнителя

#@markdown Выберете максимальное количество композиций
max_song = 80 #@param {type:"slider", min:20, max:80, step:1}

#@markdown Напишите исполнителя
artist_name = "ZeroPeople" #@param {type:"string"}

#@markdown ### Тыкните на кнопку play :)

!pip install lyricsgenius &> /dev/null

import lyricsgenius
import re

genius = lyricsgenius.Genius("YOUR KEY")
artist = genius.search_artist(artist_name, max_songs=max_song)

train_name = artist_name
val_name = f"{artist_name}_v"
dir_name = artist_name

def clear_lyrics(text):
  text = re.sub("[\(\[].*?[\)\]]", '', text)
  lines = text.split("\n")
  non_empty_lines = [line for line in lines if line.strip() != ""]
  string_without_empty_lines = ""
  for line in non_empty_lines:
      string_without_empty_lines += line + "\n"
  return " ".join(string_without_empty_lines.split(" ")[:-2])

with open(train_name, "w") as myfile:
    for song in artist.songs[:-8]:
      text = clear_lyrics(song.lyrics).strip()
      myfile.write(text)

with open(val_name, "w") as myfile:
    for song in artist.songs[-7:]:
      text = clear_lyrics(song.lyrics)
      myfile.write(text)

In [ ]:
#@title Дообучение модели. Тут будет долго, поэтому без чая не обойтись.
#@markdown В качестве ускорителя нужно выбрать GPU. Среда выполнения -> Сменить среду выполнения -> Аппаратный ускоритель -> GPU

epoch_count = 20


!pip install torch==1.4.0 &> /dev/null
!pip3 install transformers==3.5.0 &> /dev/null
!git clone  https://github.com/sberbank-ai/ru-gpts &> /dev/null
!mkdir models/

!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/{artist_name} \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file={train_name} \
    --do_eval \
    --eval_data_file={val_name} \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs {epoch_count} \
    --block_size 1024 \
    --overwrite_output_dir

In [31]:
%%capture
#@title Вводим значения для генерации

#@markdown Как должна начинаться песня?
start = "\u0416\u0438\u043B\u0438 \u0431\u044B\u043B\u0438" #@param {type:"string"}

#@markdown Введите рандомное число для случайности
seed_input = 389 #@param {type:"slider", min:10, max:1000, step:1}

#@markdown Сколько символов должно быть в сгенерированном тексте?
max_len = 50 #@param {type:"slider", min:42, max:500, step:1}

#@markdown Сделать текст креативным (всратым)?
vsrato = True #@param {type:"boolean"}

#@markdown Наказывать ли за повторения? Если нет, то выйдет типичная попса
punishment = True #@param {type:"boolean"}

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import torch
np.random.seed(seed_input)
torch.manual_seed(seed_input)
tok = GPT2Tokenizer.from_pretrained(f"models/{dir_name}")
model = GPT2LMHeadModel.from_pretrained(f"models/{dir_name}")
inpt = tok.encode(start, return_tensors="pt")

temperature = 0.87 if not vsrato else 1
penalty = 1.1 if not punishment else 5.0



model.cuda()
out = model.generate(inpt.cuda(), max_length=max_len, repetition_penalty=penalty, do_sample=True, top_k=50, top_p=0.7, temperature=temperature)

In [32]:
#@title Тык! ^_^
import textwrap

s = tok.decode(out[0])
print(textwrap.fill(s, 40))

Жили были два дурака И жили себе долго и
счастливо,      И умерли в один день.
Плёнку склеил да домой отправил... (с)
Владимир Высоцкий "Слуга без страха"
("Пианино") 1958
